In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import requests

In [2]:
!ls

Allan_Adding CPI to dataset.ipynb
Allan_Cleaned Food Prices Grouped.ipynb
Allan_Cleaned Food Prices.ipynb
Allan_Data_ Exploration.ipynb
Allan_NZ_prices.ipynb
NZ_food_prices_2020.csv
adrian pre-modeling-modified_by_Allan_for_NN.ipynb
adrian pre-modeling.ipynb
adrian-nz-types-prices_Allan.ipynb
adrian_food_global_pricing.ipynb
adrian_food_global_pricing_v2.ipynb
adrian_nutrients.ipynb
cpi_short.csv
hdi_short.csv


### API food

In [3]:
def nutrients_redux(food):
    
    product=food
   
    
    API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'
    
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage': 1,
        #'dataType': "Foundation"
    }
    response = requests.get(' https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    protein = 'not found'
    fat = 'not found'
    carb = 'not found'
    sugar = 'not found'
    sodium = 'not found'
    cholesterol = 'not found'
    calcium = 'not found'
    energy = 'not found'
    size = 'not found'
    unit = 'not found'
    try:
        
        for element in response['foods'][0]['foodNutrients']:
            if element['nutrientName'] == 'Protein':
                protein = element['value']
            if element['nutrientName'] == 'Total lipid (fat)':
                fat = element['value']
            if element['nutrientName'] == 'Carbohydrate, by difference':
                carb = element['value']
            if element['nutrientName'] == 'Sugars, Total NLEA':
                sugar = element['value']
            if element['nutrientName'] == 'Sodium, Na':
                sodium = element['value']
            if element['nutrientName'] == 'Cholesterol':
                cholesterol = element['value']
            if element['nutrientName'] == 'Calcium, Ca':
                calcium = element['value']
            if element['nutrientName'] == 'Energy':
                energy = element['value']
        size = response['foods'][0]['servingSize']
        unit = response['foods'][0]['servingSizeUnit']
    except:
        print(f'missing branded info for {food}')
    
    result = {
        'protein': protein,
        'fat': fat,
        'carb': carb,
        'sugar': sugar,
        'sodium': sodium,
        'cholesterol': cholesterol,
        'calcium': calcium,
        'kcal': energy,
        'size': size,
        'unit': unit
    }
    return result

In [4]:
def nutrients_redux_2(food):
    
    product=food
   
    
    API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'
    
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage': 1,
        'dataType': "Foundation"
    }
    response = requests.get(' https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    protein = 'not found'
    fat = 'not found'
    carb = 'not found'
    sugar = 'not found'
    sodium = 'not found'
    cholesterol = 'not found'
    calcium = 'not found'
    energy = 'not found'
    size = 'not found'
    unit = 'not found'
    try:
        
        for element in response['foods'][0]['foodNutrients']:
            if element['nutrientName'] == 'Protein':
                protein = element['value']
            if element['nutrientName'] == 'Total lipid (fat)':
                fat = element['value']
            if element['nutrientName'] == 'Carbohydrate, by difference':
                carb = element['value']
            if element['nutrientName'] == 'Sugars, Total NLEA':
                sugar = element['value']
            if element['nutrientName'] == 'Sodium, Na':
                sodium = element['value']
            if element['nutrientName'] == 'Cholesterol':
                cholesterol = element['value']
            if element['nutrientName'] == 'Calcium, Ca':
                calcium = element['value']
            if element['nutrientName'] == 'Energy':
                energy = element['value']
        size = response['foods'][0]['servingSize']
        unit = response['foods'][0]['servingSizeUnit']
    except:
        print(f'missing foundation info for {food}')
    
    result = {
        'protein': protein,
        'fat': fat,
        'carb': carb,
        'sugar': sugar,
        'sodium': sodium,
        'cholesterol': cholesterol,
        'calcium': calcium,
        'kcal': energy,
        'size': size,
        'unit': unit
    }
    return result

In [5]:
#a function to call the other two functions if the first fails
def nutrients_super(food):
    product=food
    
    result = nutrients_redux_2(product)
    
    if sum(x== 'not found' for x in result.values()) >= 6:
        result = nutrients_redux(product)
        print('switched to branded')
    else:
        print('foundation')
    return result

### Getting the table NZ

In [6]:
nz_df = pd.read_csv('../raw_data/NZ_food_prices_2020.csv')
nz_df.head()

,Unnamed: 0,Series_reference,Period,Data_value,STATUS,UNITS,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Unit
0,163,CPIM.SAP0100,2020.01,4.27,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Oranges,1kg,0,1kg
1,346,CPIM.SAP0101,2020.01,3.00,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Bananas,1kg,0,1kg
2,529,CPIM.SAP0102,2020.01,4.62,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Apples,1kg,0,1kg
3,712,CPIM.SAP0103,2020.01,7.78,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Kiwifruit,1kg,0,1kg
4,895,CPIM.SAP0104,2020.01,2.23,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Sultanas (supermarket only),375g,0,375g


In [7]:
nz_df['Series_title_1'].unique()

array(['Oranges', 'Bananas', 'Apples', 'Kiwifruit',
       'Sultanas (supermarket only)',
       'Peaches - canned (supermarket only)', 'Lettuce', 'Broccoli',
       'Cabbage', 'Tomatoes', 'Carrots', 'Mushrooms', 'Potatoes',
       'Peas - frozen (supermarket only)', 'Beef steak - blade',
       'Beef steak - porterhouse/sirloin', 'Beef - mince',
       'Pork - loin chops', 'Lamb - chops', 'Sausages',
       'Tuna - canned (supermarket only)', 'Biscuits - chocolate',
       'Breakfast biscuits', 'Flour - white (supermarket only)',
       'Rice - long grain', 'Milk - standard homogenised',
       'Yoghurt - flavoured', 'Cheese - mild cheddar (supermarket only)',
       'Eggs', 'Butter - salted', 'Sugar - white',
       'Chocolate - block (supermarket only)', 'Spaghetti - canned',
       'Coffee - instant', 'Tea bags (supermarket only)', 'Soft drink',
       'Bottled water', 'Fish and chips', 'Meat pie - hot',
       'Fruit juice - apple based (supermarket only)', 'Potato crisps',
      

In [8]:
nutrients_super('peanuts')

missing foundation info for peanuts
switched to branded


{'protein': 21.4,
 'fat': 46.4,
 'carb': 21.4,
 'sugar': 'not found',
 'sodium': 607,
 'cholesterol': 0.0,
 'calcium': 0.0,
 'kcal': 571,
 'size': 28.0,
 'unit': 'g'}

In [9]:
nz_df['type'] = nz_df['Series_title_1'].apply(lambda x: x.split('(')[0])
nz_df['type'] = nz_df['type'].apply(lambda x: x.replace('-',''))
nz_df['type'] = nz_df['type'].apply(lambda x: x.strip(' '))

nz_df['type'].head(15)

0               Oranges
1               Bananas
2                Apples
3             Kiwifruit
4              Sultanas
5       Peaches  canned
6               Lettuce
7              Broccoli
8               Cabbage
9              Tomatoes
10              Carrots
11            Mushrooms
12             Potatoes
13         Peas  frozen
14    Beef steak  blade
Name: type, dtype: object

In [10]:
clean_df = nz_df.filter(items=['type','unit','Data_value','UNITS'])

In [11]:
clean_df.isnull().sum()

type          0
Data_value    0
UNITS         0
dtype: int64

In [12]:
clean_df.value_counts()

type                 Data_value  UNITS  
Apples               4.62        Dollars    1
Peanut butter        2.87        Dollars    1
Oranges              4.27        Dollars    1
Packaged cake slice  4.00        Dollars    1
Packaged meal        2.39        Dollars    1
                                           ..
Dessert              6.56        Dollars    1
Dried mixed herbs    2.60        Dollars    1
Dried pasta          1.91        Dollars    1
Drinking chocolate   4.13        Dollars    1
Yoghurt  flavoured   5.18        Dollars    1
Length: 155, dtype: int64

In [13]:
food_types = pd.DataFrame(clean_df['type'].unique(), columns=['food_type'])

In [14]:
food_types

,food_type
0,Oranges
1,Bananas
2,Apples
3,Kiwifruit
4,Sultanas
...,...
136,Mayonnaise
137,Dried mixed herbs
138,Peanuts
139,Ham


In [15]:
food_types['protein'] = None
food_types['fat'] = None
food_types['carb'] = None
food_types['sugar'] = None
food_types['sodium'] = None
food_types['cholesterol'] = None
food_types['calcium'] = None
food_types['kcal'] = None
food_types['size'] = None
food_types['unit'] = None

### Applying the API to the clean nz table

In [16]:
#applying the nutrients_super
for index, row in food_types.iterrows():
    food_nutrients = nutrients_super(row['food_type'])
    food_types.loc[index, 'protein'] = food_nutrients['protein']
    food_types.loc[index, 'fat'] = food_nutrients['fat']
    food_types.loc[index, 'carb'] = food_nutrients['carb']
    food_types.loc[index, 'sugar'] = food_nutrients['sugar']
    food_types.loc[index, 'cholesterol'] = food_nutrients['cholesterol']
    food_types.loc[index, 'sodium'] = food_nutrients['sodium']
    food_types.loc[index, 'calcium'] = food_nutrients['calcium']
    food_types.loc[index, 'kcal'] = food_nutrients['kcal']
    food_types.loc[index, 'size'] = food_nutrients['size']
    food_types.loc[index, 'unit'] = food_nutrients['unit']

missing foundation info for Oranges
foundation
missing foundation info for Bananas
foundation
missing foundation info for Apples
foundation
missing foundation info for Kiwifruit
foundation
missing foundation info for Sultanas
switched to branded
missing foundation info for Peaches  canned
foundation
missing foundation info for Lettuce
foundation
missing foundation info for Broccoli
foundation
missing foundation info for Cabbage
missing branded info for Cabbage
switched to branded
missing foundation info for Tomatoes
foundation
missing foundation info for Carrots
foundation
missing foundation info for Mushrooms
foundation
missing foundation info for Potatoes
switched to branded
missing foundation info for Peas  frozen
foundation
missing foundation info for Beef steak  blade
foundation
missing foundation info for Beef steak  porterhouse/sirloin
missing branded info for Beef steak  porterhouse/sirloin
switched to branded
missing foundation info for Beef  mince
foundation
missing foundatio

missing foundation info for Tea
switched to branded
missing foundation info for Vinegar
missing branded info for Vinegar
switched to branded
missing foundation info for Wheatmeal bread
foundation
missing foundation info for Wholegrain bread
foundation
missing foundation info for Fresh herbs
foundation
missing foundation info for Olives
switched to branded
missing foundation info for Margarine/table spread
missing branded info for Margarine/table spread
switched to branded
missing foundation info for Mayonnaise
switched to branded
missing foundation info for Dried mixed herbs
foundation
missing foundation info for Peanuts
switched to branded
missing foundation info for Ham
foundation
missing foundation info for Chewing gum
missing branded info for Chewing gum
switched to branded


In [17]:
food_types

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit
0,Oranges,0.91,0.15,11.8,8.57,9.0,not found,43.0,196,not found,not found
1,Bananas,0.73,0.22,20.1,15.8,not found,not found,not found,357,not found,not found
2,Apples,0.148,0.162,15.7,13.3,1.01,not found,5.98,not found,not found,not found
3,Kiwifruit,1.06,0.44,14.0,8.99,5.0,not found,35.0,241,not found,not found
4,Sultanas,2.5,0.0,77.5,not found,0.0,0.0,50.0,325,40.0,g
...,...,...,...,...,...,...,...,...,...,...,...
136,Mayonnaise,0.0,75.0,0.0,not found,667,83.0,0.0,750,12.0,g
137,Dried mixed herbs,79.9,0.65,6.02,not found,1250.0,17.0,104,376,not found,not found
138,Peanuts,25.0,53.6,17.9,not found,393,0.0,0.0,607,28.0,g
139,Ham,19.6,3.68,2.36,2.2,1030.0,not found,6.0,507,not found,not found


In [18]:
food_types['unit'].value_counts()/len(food_types)

not found    0.751773
g            0.226950
ml           0.021277
Name: unit, dtype: float64

In [19]:
#applying the nutrients_redux
food_types_redux = pd.DataFrame(clean_df['type'].unique(), columns=['food_type'])
for index, row in food_types_redux.iterrows():
    food_nutrients = nutrients_redux(row['food_type'])
    food_types_redux.loc[index, 'protein'] = food_nutrients['protein']
    food_types_redux.loc[index, 'fat'] = food_nutrients['fat']
    food_types_redux.loc[index, 'carb'] = food_nutrients['carb']
    food_types_redux.loc[index, 'sugar'] = food_nutrients['sugar']
    food_types_redux.loc[index, 'cholesterol'] = food_nutrients['cholesterol']
    food_types_redux.loc[index, 'sodium'] = food_nutrients['sodium']
    food_types_redux.loc[index, 'calcium'] = food_nutrients['calcium']
    food_types_redux.loc[index, 'kcal'] = food_nutrients['kcal']
    food_types_redux.loc[index, 'size'] = food_nutrients['size']
    food_types_redux.loc[index, 'unit'] = food_nutrients['unit']

missing branded info for Bananas
missing branded info for Peaches  canned
missing branded info for Lettuce
missing branded info for Cabbage
missing branded info for Peas  frozen
missing branded info for Beef steak  blade
missing branded info for Beef steak  porterhouse/sirloin
missing branded info for Beef  mince
missing branded info for Pork  loin chops
missing branded info for Lamb  chops
missing branded info for Sausages
missing branded info for Tuna  canned
missing branded info for Biscuits  chocolate
missing branded info for Flour  white
missing branded info for Rice  long grain
missing branded info for Milk  standard homogenised
missing branded info for Cheese  mild cheddar
missing branded info for Butter  salted
missing branded info for Sugar  white
missing branded info for Chocolate  block
missing branded info for Spaghetti  canned
missing branded info for Coffee  instant
missing branded info for Bottled water
missing branded info for Fish and chips
missing branded info for Mea

In [20]:
food_types_redux['protein'].value_counts()/len(food_types)

0.0          0.141844
not found    0.021277
2.5          0.021277
1.1          0.021277
5.42         0.014184
               ...   
11.1         0.007092
3.57         0.007092
0.2          0.007092
5.11         0.007092
16.1         0.007092
Name: protein, Length: 106, dtype: float64

In [21]:
len(food_types_redux['sugar'].loc[food_types_redux['sugar']== 'not found'])

141

In [22]:
for nutrient in food_types_redux:
    missing = len(food_types_redux[nutrient].loc[food_types_redux[nutrient]== 'not found'])
    
    print(nutrient, 'is missing', missing/len(food_types_redux))

food_type is missing 0.0
protein is missing 0.02127659574468085
fat is missing 0.028368794326241134
carb is missing 0.028368794326241134
sugar is missing 1.0
cholesterol is missing 0.05673758865248227
sodium is missing 0.028368794326241134
calcium is missing 0.0851063829787234
kcal is missing 0.02127659574468085
size is missing 0.574468085106383
unit is missing 0.574468085106383


In [23]:
for nutrient in food_types:
    missing = len(food_types[nutrient].loc[food_types[nutrient]== 'not found'])
    
    print(nutrient, 'is missing', missing/len(food_types), 'percent')

food_type is missing 0.0 percent
protein is missing 0.028368794326241134 percent
fat is missing 0.028368794326241134 percent
carb is missing 0.03546099290780142 percent
sugar is missing 0.6312056737588653 percent
sodium is missing 0.0425531914893617 percent
cholesterol is missing 0.4326241134751773 percent
calcium is missing 0.07801418439716312 percent
kcal is missing 0.0851063829787234 percent
size is missing 0.75177304964539 percent
unit is missing 0.75177304964539 percent


In [24]:
clean_df.shape

(155, 3)

### Exploring responses

In [25]:
#basic manual api call
API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'

product='fish'

params = {
    'api_key': API_KEY,
    'query': product,
    'pageSize': 5
}

In [26]:
try:
    product = 'Ham'
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage':1,
        'dataType': "Foundation"
    }
    response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    print(response['foods'][0]['servingSize'], response['foods'][0]['servingSizeUnit'])
except:
    print('not found')

not found


In [27]:
response['foods'][0].keys()

dict_keys(['fdcId', 'description', 'lowercaseDescription', 'commonNames', 'additionalDescriptions', 'dataType', 'ndbNumber', 'publishedDate', 'foodCategory', 'mostRecentAcquisitionDate', 'allHighlightFields', 'score', 'foodNutrients', 'finalFoodInputFoods', 'foodMeasures', 'foodAttributes', 'foodAttributeTypes', 'foodVersionIds'])

In [42]:
response['foods'][0]['foodNutrients'][0]

{'nutrientId': 2048,
 'nutrientName': 'Energy (Atwater Specific Factors)',
 'nutrientNumber': '958',
 'unitName': 'KCAL',
 'derivationCode': 'NC',
 'derivationDescription': 'Calculated',
 'derivationId': 49,
 'value': 126,
 'foodNutrientSourceId': 2,
 'foodNutrientSourceCode': '4',
 'foodNutrientSourceDescription': 'Calculated or imputed',
 'rank': 290,
 'indentLevel': 1,
 'foodNutrientId': 13338459}

In [43]:
response['foods']

[{'fdcId': 746952,
  'description': 'Ham, sliced, restaurant',
  'lowercaseDescription': 'ham, sliced, restaurant',
  'commonNames': '',
  'additionalDescriptions': '',
  'dataType': 'Foundation',
  'ndbNumber': 7129,
  'publishedDate': '2019-12-16',
  'foodCategory': 'Pork Products',
  'mostRecentAcquisitionDate': '2018-09-20',
  'allHighlightFields': '',
  'score': 328.3024,
  'foodNutrients': [{'nutrientId': 2048,
    'nutrientName': 'Energy (Atwater Specific Factors)',
    'nutrientNumber': '958',
    'unitName': 'KCAL',
    'derivationCode': 'NC',
    'derivationDescription': 'Calculated',
    'derivationId': 49,
    'value': 126,
    'foodNutrientSourceId': 2,
    'foodNutrientSourceCode': '4',
    'foodNutrientSourceDescription': 'Calculated or imputed',
    'rank': 290,
    'indentLevel': 1,
    'foodNutrientId': 13338459},
   {'nutrientId': 2047,
    'nutrientName': 'Energy (Atwater General Factors)',
    'nutrientNumber': '957',
    'unitName': 'KCAL',
    'derivationCode': '